<a href="https://colab.research.google.com/github/CurtesMalteser/text-generator-rnn/blob/master/text-generator-rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

### Load Data

In [21]:
with open('drive/My Drive/anna.txt', 'r') as f:
    text = f.read()

text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'